# Exercise 4
Task:
- Implement DMDc as shown in Example 3.6
- Test script on data provided in handout 4
- Modify number of singular values and analyze effect on approximation quality





In [1]:
import numpy as np
import matplotlib.pyplot as plt

from utils import utils

%matplotlib inline

/home/matthias/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [ ]:
def dynamic_mode_decomposition(x, x_prime,r):
    """Implementation of the dynamic mode decomposition algorithm.

    Args:
        x (numpy.ndarray): The data matrix.
        x_prime (numpy.ndarray): The data matrix shifted by one timestep.
        r (int): The rank of the reduced order model.

    Returns:
        numpy.ndarray: The DMD modes.
        numpy.ndarray: The eigenvalues.
    """
    # 1. compute SVD of x
    u, s, v_h = np.linalg.svd(x, full_matrices=False)
    
    # 2. truncate the SVD to rank r
    u_tilde = u[:, :r]
    s_tilde = np.diag(s[:r])
    v_h_tilde = v_h[:r, :]
    
    # 3. compute A_tilde -> reduced order operator
    u_h_tilde = u_tilde.conj().T
    v_sin_v = np.linalg.solve(s_tilde.conj().T,v_h_tilde).conj().T
    a_tilde = u_h_tilde @ x_prime @ v_sin_v

    # 4. compute eigenvalues and eigenvectors of A_tilde
    eigenvalues, eigenvectors = np.linalg.eig(a_tilde)

    # 5. compute DMD modes
    dmd_modes = x_prime @ v_sin_v @ eigenvectors

    return dmd_modes, eigenvalues
dmd = dynamic_mode_decomposition


In [2]:
def dynamic_mode_decomposition_control(x, x_prime, u, r):
    """ Implementation of dynamic mode decomposition with control.

    Args:
        x (numpy.ndarray): The data matrix.
        x_prime (numpy.ndarray): The data matrix shifted by one timestep.
        u (numpy.ndarray): The control matrix.
        r (int): The rank of the reduced order model.

    Returns:
        numpy.ndarray: The DMD modes.
        numpy.ndarray: The eigenvalues.
        numpy.ndarray: The reduced order input matrix.
    """
    omega = np.concatenate((x, u), axis=0)
    
    u_, s, v_h = np.linalg.svd(omega, full_matrices=False)

    u_tilde = u_[:, :r]
    s_tilde = np.diag(s[:r])
    v_h_tilde = v_h[:r, :]

    omega_pinv_tilde = np.linalg.solve(s_tilde.conj().T, v_h_tilde).conj().T
    ab_tilde = x_prime @ omega_pinv_tilde @ u_tilde

    a_tilde = ab_tilde[:, :x.shape[0]]
    b_tilde = ab_tilde[:, x.shape[0]:]

    eigenvalues, eigenvectors = np.linalg.eig(a_tilde)

    modes = x_prime @ omega_pinv_tilde @ u_tilde @ eigenvectors

    return modes, eigenvalues, b_tilde

    


dmdc = dynamic_mode_decomposition_control